In [1]:
## Auxillary functions needed for our analysis

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import RidgeCV
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

In [3]:
#Convert categorical features to numerical features by arranging them in ascending order of the target
#Inputs are pandas dataframes
def cattonum(df,dftest,feature,target):
    meanstd = np.zeros([df[feature].nunique(),2])
    for i,x in enumerate(df[feature].unique()):
        meanstd[i,0] = df[df[feature]==x][target].mean()
        meanstd[i,1] = df[df[feature]==x][target].std()
    s = meanstd[:,0].argsort()
    meanstd = meanstd[s]
    labels = df[feature].unique()[s]

    plt.errorbar(range(0,s.shape[0]), meanstd[:,0], 0.5*meanstd[:,1], marker='o', mfc='red',
         mec='blue', ms=2, mew=4)
    plt.xticks(range(0,s.shape[0]), labels, rotation='vertical')
    df[feature] = df[feature].replace(labels,range(1,s.shape[0]+1))
    dftest[feature] = dftest[feature].replace(labels,range(1,s.shape[0]+1))
    return None

In [4]:
def cattonum1(df,dftest,feature,target):
    meanstd = np.zeros([df[feature].nunique(),2])
    for i,x in enumerate(df[feature].unique()):
        meanstd[i,0] = df[df[feature]==x][target].mean()
        meanstd[i,1] = df[df[feature]==x][target].std()
    s = meanstd[:,0].argsort()
    meanstd = meanstd[s]
    labels = df[feature].unique()[s]

#    plt.errorbar(range(1,s.shape[0]+1), meanstd[:,0], 0.5*meanstd[:,1], marker='o', mfc='red',
#         mec='blue', ms=2, mew=4)
    plt.plot([1,s.shape[0]], [meanstd[0,0],meanstd[-1,0]], 'black', linestyle='--', marker='')
    plt.xticks(range(1,s.shape[0]+1), labels, rotation='vertical')
    eng_lab = 1+(meanstd[:,0]-meanstd[0,0])*((s.shape[0]-1)/(meanstd[-1,0]-meanstd[0,0]))
    #print(eng_lab)
    #df[feature] = df[feature].replace(labels,range(1,s.shape[0]+1))
    df[feature] = df[feature].replace(labels,eng_lab)
    plt.errorbar(eng_lab, meanstd[:,0], 0.5*meanstd[:,1], marker='o', mfc='red',
         mec='blue', ms=2, mew=4)
    dftest[feature] = dftest[feature].replace(labels,eng_lab)
    
    
    return None

In [5]:
# Let me start with data first  that is numerical continuous and not categorical.An easy way to do that is to find nunique
def dropcategorical(df,dftest,n):
    #drops columns which have no. of features less than n
    df_copy = df.copy()
    dftest_copy = dftest.copy()
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64','uint8']
    df_copy = df_copy.select_dtypes(numerics)
    dftest_copy = dftest_copy.select_dtypes(numerics)
    l = []
    for i,x in enumerate(df_copy.nunique()):
        if x < n:
            l.append(df_copy.columns[i])
    df_copy.drop(l,axis=1,inplace=True)
    dftest_copy.drop(l,axis=1,inplace=True)
    return df_copy,dftest_copy,l

In [8]:
# Now let us start systematically starting w/ features strongly corellated w/ salesprice and work our way through. I will use
# sklearn for this process
def withncontfeatures(train,test,n):
    if n > train.shape[1]-1:
        print('too many features')
        return
    corrmat = train.corr()
    k = train_cont.shape[1] #number of variables for heatmap
    cols = corrmat.nlargest(k, 'SalePrice')['SalePrice'].index
    cm = np.corrcoef(train_cont[cols].values.T)
    l = cols[np.argsort(np.abs(cm[0]))[::-1][1:n+1]].values#n+1 because just 1:1 gives the index only
#    print(l[-1])
    trainX = train[l]
    trainX = (trainX-trainX.mean(axis=0))/trainX.std(axis=0)
#    trainy = train.iloc[:,[-1]].values
    trainy = train['SalePrice'].values
    
    testX = test[l]
    testX = (testX-testX.mean(axis=0))/testX.std(axis=0)

    a = np.logspace(0,3,100)
    ridge = RidgeCV(alphas=a).fit(trainX, trainy)
    pred = ridge.predict(trainX)
    testpred = ridge.predict(testX)
    alpha = ridge.alpha_

    pred[pred < 0] = np.mean(trainy)
    err = np.sqrt(((pred-trainy)**2).sum()/len(trainy))
    return err,alpha,l,testpred,pred